<a href="https://colab.research.google.com/github/shugen002/FindMyself/blob/master/%E5%AF%BB%E6%89%BE%E6%A0%91%E6%A0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 0、事情起因

B站上一位“誰もぢ好き_Official”的同学发布了一张超多V的头像的大拼图，然而我却找不到自己的头像，所以我就发挥自己的一点点能力，写下了这段代码。

![](https://i0.hdslb.com/bfs/album/f4a88292542fae9e100faf2921f07a9362ffeaf0.jpg)

### 0.5、用法 及 缺陷

第一次运行时，请依次点击步骤1、步骤2的代码块的运行按钮

然后及后续运行时只需要点击步骤4的代码块即可

注意，底边和右边的图块因为大小不一样，所以未做搜索

### 1、安装工具库


In [ ]:
!pip install image_similarity_measures

### 2、下载并切割图像


In [ ]:
from PIL import Image
import os
import urllib.request

urllib.request.urlretrieve('https://i0.hdslb.com/bfs/album/f4a88292542fae9e100faf2921f07a9362ffeaf0.jpg',"image.jpg")

# 原始图片文件名
infile = "image.jpg"

img = Image.open(infile)
width, height = img.size
# 切割的图块大小，后续如果更新了的话，请务必注意图块大小是否变化
chopsizew = 126
chopsizeh = 94

if not os.path.exists("img"):
  os.mkdir("img")

# Save Chops of original image
for x0 in range(0, width, chopsizew):
   for y0 in range(0, height, chopsizeh):
      box = (x0, y0,
             x0+chopsizew if x0+chopsizew <  width else  width - 1,
             y0+chopsizeh if y0+chopsizeh < height else height - 1)
      img.crop(box).save('img/x%03d.y%03d.jpg' % (x0/chopsizew, y0/chopsizeh))
print ("切割完成")

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


切割完成


### 3、上传搜索目标

将目标图像最长边缩放为图块最长边，然后裁剪出图块大小的图片，也就是126x94


### 4、开始搜索

In [ ]:
import json
import os
import time
import base64

import cv2
import numpy as np

from IPython.display import HTML, display

import image_similarity_measures
from image_similarity_measures.quality_metrics import metric_functions

# 输入文件
targetFile = "target.jpg"

metrics = [
  {"name":"psnr","reverse": True},
  {"name":"rmse","reverse": False},
  # 好像效果不怎么样，屏蔽
  # {"name":"sam","reverse": True},
  {"name":"sre","reverse": True},
  {"name":"ssim","reverse": True},
  # 不知道为什么全是0，屏蔽
  # {"name":"issm","reverse": True},
  # 这两个太慢了，需要的自己开吧
  # {"name":"fsim","reverse": True},
  # {"name":"uiq","reverse": True}
]

def progress(value, max=100):
  return HTML("""
    <progress
        value='{value}'
        max='{max}',
        style='width: 80%'
    >
        {value}
    </progress>
  """.format(value=value, max=max))

def read_image(path):
  return cv2.imread(path)

def evaluation(org_img, pred_img_path, metric):
  pred_img = read_image(pred_img_path)
  metric_func = metric_functions[metric]
  out_value = float(metric_func(org_img, pred_img))
  return out_value

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def showTable(items):
  chunk = chunks(items,8)
  rows = list(
      map(lambda x: "<tr>"+"".join(x)+"</tr>" , chunk)
      )

  body="".join(rows)
  return HTML("""
    <table>
      <tbody>
        {body}
      </tbody>
    </table>
  """.format(body=body))

def getImage(path):
  imageFile = open(path, "rb")
  return """
    <td>
      <img src="data:image;base64,{base64}">
      <p>{filename}</p>
    </td>
  """.format(base64=base64.b64encode(imageFile.read()).decode('utf-8'),filename=path)

def getImageWithNum(path,num):
  imageFile = open(path, "rb")
  return """
    <td>
      <img src="data:image;base64,{base64}">
      <p>{filename}</p>
      <p>{num}</p>
    </td>
  """.format(base64=base64.b64encode(imageFile.read()).decode('utf-8'), filename=path, num=str(num)) 

def getResult(elem):
  return elem["result"]

def main():
  files = os.listdir("img")
  org_img = read_image(targetFile)
  skipped = []
  for metric in metrics:
    result = []
    print ("Start " + metric["name"] + "...")
    starttime = time.time()
    total = len(files)
    out = display(progress(0, total), display_id=True)
    for index in range(len(files)):
      filename = files[index]
      out.update(progress(index+1, total))
      try:
        result.append({
          "target": filename,
          "result": evaluation(
            org_img,
            pred_img_path = "img/" + filename,
            metric = metric["name"]
          )
        })
      except:
        skipped.append(filename)
        pass
    print("Finished " + metric["name"] + " with time " + "{:.5f}".format(time.time()-starttime))
    result.sort(key=getResult,reverse=metric["reverse"]);
    open("result_" + metric["name"] + ".json", 'w').write(json.dumps(result))
    open("skipped_" + metric["name"] + ".json", 'w').write(json.dumps(skipped))
    first20=[]
    for i in range(20):
      first20.append(getImageWithNum("img/"+result[i]["target"],"{:.5f}".format(result[i]["result"])))
    display(showTable(first20))

if __name__ == "__main__":
  main()


Start psnr...


Finished psnr with time 9.81056


Start rmse...


Finished rmse with time 12.75248


Start sre...


Finished sre with time 15.22877


Start ssim...


Finished ssim with time 27.15593


### ∞、清理

In [5]:
!rm -rf result_*.json
!rm -rf skipped_*.json